In [1]:
def remove_initial_offset(df, columns):
    """
    Removes the initial offset for specified columns in a Pandas DataFrame.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame containing the data.
        columns (list of str): List of column names to adjust.
    
    Returns:
        pd.DataFrame: A new DataFrame with the initial offsets removed.
    """
    # Ensure a copy of the DataFrame to avoid modifying the original
    adjusted_df = df.copy()
    
    # Compute the initial offset for each column
    initial_offsets = adjusted_df.iloc[0][columns]
    
    # Subtract the initial offsets from the specified columns
    for col in columns:
        adjusted_df[col] -= initial_offsets[col]
    
    return adjusted_df

def calculate_derivative(df, columns, periods=1, prefix="d"):
    """
    Calculates the derivative (difference) for specified columns in a DataFrame.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame containing the data.
        columns (list of str): List of column names to calculate the derivative for.
        periods (int): Number of periods to shift for calculating differences. Default is 1.
        prefix (str): Prefix to add to the column names for the derivative columns. Default is "d".
    
    Returns:
        pd.DataFrame: A new DataFrame with added derivative columns.
    """
    # Ensure a copy of the DataFrame to avoid modifying the original
    derivative_df = df.copy()
    
    # Calculate the derivative for each specified column
    for col in columns:
        derivative_df[f"{prefix}{col}"] = derivative_df[col].diff(periods=periods)
        derivative_df[f"{prefix}{col}"][0] = 0
    
    return derivative_df

from scipy.signal import butter, filtfilt

def apply_filter(df, columns, filter_type="moving_average", window_size=5, cutoff=0.1, order=2, **kwargs):
    """
    Applies a filter to specified columns in a DataFrame.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        columns (list of str): The list of column names to filter.
        filter_type (str): The type of filter to apply. 
            Supported: "moving_average", "exponential_moving_average", "low_pass_butterworth".
        window_size (int): The window size for moving average or exponential moving average.
        cutoff (float): Cutoff frequency for Butterworth low-pass filter (normalized between 0 and 0.5).
        order (int): The order of the Butterworth filter.
        **kwargs: Additional arguments for extensibility.
    
    Returns:
        pd.DataFrame: A new DataFrame with filtered columns.
    """
    filtered_df = df.copy()
    
    if filter_type == "moving_average":
        for col in columns:
            filtered_df[f"{col}"] = filtered_df[col].rolling(window=window_size, **kwargs).mean()

    elif filter_type == "exponential_moving_average":
        for col in columns:
            filtered_df[f"{col}"] = filtered_df[col].ewm(span=window_size, **kwargs).mean()

    elif filter_type == "low_pass_butterworth":
        # Design Butterworth filter
        b, a = butter(order, cutoff, btype='low', analog=False)
        for col in columns:
            filtered_df[f"{col}"] = filtfilt(b, a, filtered_df[col].fillna(method='ffill').fillna(0))
    else:
        raise ValueError(f"Unsupported filter type: {filter_type}")
    
    return filtered_df

def detect_turn_end(df, threshold=2, duration=2):
    """
    Detects the end of a turn based on derivative values (dy) staying below a threshold for a certain duration.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame with a "dy" column.
        threshold (float): The threshold value below which "dy" indicates no turning.
        duration (float): Duration in seconds for which "dy" should stay below the threshold to detect end of turn.

    Returns:
        pd.DataFrame: DataFrame with an added "turning" column (1 if turning, 0 otherwise).
    """
    df = df.copy()
    df["turning"] = 1  # Assume turning by default

    # Ensure "ts_seconds" exists for calculating duration
    if "ts_seconds" not in df.columns:
        raise ValueError("The DataFrame must include a 'ts_seconds' column for timing information.")

    # Create a boolean column for values below the threshold
    df["below_threshold"] = (df["dy"].abs() < threshold).astype(int)

    # Calculate rolling sums over the duration to check if below threshold consistently
    sampling_rate = 1 / df["ts_seconds"].diff().median()  # Calculate sampling rate
    window_size = int(duration * sampling_rate)  # Convert duration to number of samples

    # Check if below_threshold is 1 for the entire window
    df["below_threshold_rolling"] = df["below_threshold"].rolling(window=10, min_periods=1).mean()

    # Mark "turning" as 0 if below_threshold_rolling stays below 1 (indicating turn end)
    df.loc[df["below_threshold_rolling"] == 1, "turning"] = 0

    # Drop helper columns for cleanliness if desired
    display(df)
    # df = df.drop(columns=["below_threshold", "below_threshold_rolling"])

    return df

In [2]:
import pandas as pd
import numpy as np
import os

%matplotlib qt
import matplotlib.pyplot as plt

files = ["turn_right.csv", "turn_left.csv"]#, "right_lane.csv", "left_lane.csv"]
# files = ["test1.csv"]

for filename in files:

    columns = ["ts", "y", "p", "r"]
    df = pd.read_csv(os.path.join("logs",filename), skiprows=1, names=columns)
    
    df["ts"] = pd.to_datetime(df["ts"])
    # Calculate seconds relative to the first timestamp
    df["ts_seconds"] = (df["ts"] - df["ts"].iloc[0]).dt.total_seconds()

    # Unwrap angular data
    threshold = 180
    df["y"] = np.unwrap(df["y"], discont=threshold)
    df["p"] = np.unwrap(df["p"], discont=threshold)
    df["r"] = np.unwrap(df["r"], discont=threshold)
    df = remove_initial_offset(df, ["y", "p", "r"])
    
    # df = apply_filter(df, ["y", "p", "r"], filter_type="low_pass_butterworth")
    
    df = calculate_derivative(df, ["y", "p", "r"])

    df = detect_turn_end(df, threshold=1, duration=1)

    # Calculate timestamp differences to detect inconsistencies
    df["ts_diff"] = df["ts_seconds"].diff()

    # Plot data
    plt.figure()
    plt.suptitle(filename)
    
    # Plot raw data
    plt.subplot(3, 1, 1)
    plt.plot(df["ts_seconds"], df[["y", "p", "r"]])
    plt.legend(["y", "p", "r"])
    plt.ylabel("Angles (deg)")
    
    # Plot derivatives
    plt.subplot(3, 1, 2)
    plt.plot(df["ts_seconds"], df[["dy", "dp", "dr"]])
    plt.plot(df["ts_seconds"], df[["turning"]])
    plt.legend(["dy", "dp", "dr"])
    plt.ylabel("Angular Rates (deg/s)")
    
    # # Plot timestamp inconsistencies
    plt.subplot(3, 1, 3)
    plt.stem(df["ts_seconds"], df["ts_diff"], label="Time Diff")
    plt.legend()
    plt.xlabel("Time (s)")
    plt.ylabel("Time Difference (s)")
    plt.grid()

plt.show()


C:\Users\alixd\AppData\Local\Temp\ipykernel_11456\555421578.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derivative_df[f"{prefix}{col}"][0] = 0
C:\Users\alixd\AppData\Local\Temp\ipykernel_11456\555421578.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derivative_df[f"{prefix}{col}"][0] = 0
C:\Users\alixd\AppData\Local\Temp\ipykernel_11456\555421578.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derivative_df[f"{prefi

,ts,y,p,r,ts_seconds,dy,dp,dr,turning,below_threshold,below_threshold_rolling
0,2025-01-20 13:56:18.825740,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0,1,1.0
1,2025-01-20 13:56:18.840926,-0.01,0.00,0.01,0.015186,-0.01,0.00,0.01,0,1,1.0
2,2025-01-20 13:56:18.841938,-0.02,-0.01,0.02,0.016198,-0.01,-0.01,0.01,0,1,1.0
3,2025-01-20 13:56:18.842938,-0.04,-0.01,0.02,0.017198,-0.02,0.00,0.00,0,1,1.0
4,2025-01-20 13:56:18.870801,-0.06,0.00,0.01,0.045061,-0.02,0.01,-0.01,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
93,2025-01-20 13:56:26.506669,87.67,7.69,-2.00,7.680929,-0.04,-0.04,0.00,1,1,0.9
94,2025-01-20 13:56:26.508669,87.64,7.67,-1.97,7.682929,-0.03,-0.02,0.03,1,1,0.9
95,2025-01-20 13:56:26.625747,87.54,7.41,-1.98,7.800007,-0.10,-0.26,-0.01,1,1,0.9
96,2025-01-20 13:56:26.627028,87.56,7.35,-1.95,7.801288,0.02,-0.06,0.03,1,1,0.9


C:\Users\alixd\AppData\Local\Temp\ipykernel_11456\555421578.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derivative_df[f"{prefix}{col}"][0] = 0
C:\Users\alixd\AppData\Local\Temp\ipykernel_11456\555421578.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derivative_df[f"{prefix}{col}"][0] = 0
C:\Users\alixd\AppData\Local\Temp\ipykernel_11456\555421578.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derivative_df[f"{prefi

,ts,y,p,r,ts_seconds,dy,dp,dr,turning,below_threshold,below_threshold_rolling
0,2025-01-20 13:55:44.358960,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0,1,1.0
1,2025-01-20 13:55:44.478462,-0.01,0.00,0.00,0.119502,-0.01,0.00,0.00,0,1,1.0
2,2025-01-20 13:55:44.688589,-0.02,0.00,0.01,0.329629,-0.01,0.00,0.01,0,1,1.0
3,2025-01-20 13:55:44.717773,-0.02,-0.01,0.01,0.358813,0.00,-0.01,0.00,0,1,1.0
4,2025-01-20 13:55:44.807904,-0.03,-0.01,0.01,0.448944,-0.01,0.00,0.00,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
88,2025-01-20 13:55:52.217880,-98.77,-5.78,-3.02,7.858920,21.18,-9.24,3.06,1,0,0.6
89,2025-01-20 13:55:52.278083,-98.38,-5.80,-2.95,7.919123,0.39,-0.02,0.07,1,1,0.6
90,2025-01-20 13:55:52.519675,-98.00,-5.81,-2.88,8.160715,0.38,-0.01,0.07,1,1,0.6
91,2025-01-20 13:55:52.520975,-88.77,-3.14,-0.46,8.162015,9.23,2.67,2.42,1,0,0.5
